### Create Technical Inclusion Areas, A raster based approach.
In this notebook, we create technical inclusion areas for utility scale solar or wind installations. The corresponding environmental exclusion layer and technical threshold raster layers will later be masked from this layer in order to generate the final SL1 Inclusion Area layers and subsequent area assessments. 

Make sure all inputs in the selected input geodatabase have the same projection as the template raster *prior* to running this script.  

In [1]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

#### Set input and output paths, set environmental variables for raster analyses

In [13]:
# set input and output folder
mainInputFolder = "C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data"  # enter path to input folder
mainOutputFolder = "C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\"  # enter path to output folder

# set path to input file geodatabase
input_gdb = os.path.join(mainInputFolder, "technoExclusionWind.gdb")
 
# create folder for results
results_folder = os.path.join(mainOutputFolder, "technicalInclusionAreas")

# sets environment workspace to input_gdb
arcpy.env.workspace = input_gdb

# set path to scratch folder
scratch_folder = os.path.join(mainOutputFolder, "scratch")

# set path to remap tables
remap_table_gdb = os.path.join(mainInputFolder, "remapWind.gdb") # enter path to remap table geodatabase

In [3]:
# set path to template raster
templateRaster = os.path.join(mainInputFolder, "technoExclusionWindRasters\\SRTM_250m_proj_cl.tif") ##^^ enter path to DEM data

In [4]:
# set environments for raster analyses
arcpy.env.snapRaster = templateRaster
arcpy.env.extent = templateRaster
arcpy.env.mask = templateRaster
arcpy.env.cellSize = templateRaster

In [5]:
env.workspace

'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\technoExclusionWind.gdb'

#### Define useful functions

In [6]:
# Function to perform Euclidean distance and reclassification
def euclidean_reclassify(feature_class, remap_table):
    
    # sets scratchworkspace to scratch.gdb
    arcpy.env.scratchWorkspace = scratch_folder 

    # check if scratch folder exists, if not create it
    if not os.path.exists(arcpy.env.scratchWorkspace): 
        os.makedirs(arcpy.env.scratchWorkspace)   

    # Create the euclidean raster name
    euclidean_raster_name = os.path.splitext(feature_class)[0] + "_euclidean.tif" 

    # Create Euclidean distance raster path
    euclidean_raster = os.path.join(scratch_folder, euclidean_raster_name)

    # Create the reclassified raster path
    reclassified_raster_name = os.path.splitext(feature_class)[0] + "_reclass.tif"

    # Create the reclassified raster path
    reclassified_raster_path = os.path.join(scratch_folder, reclassified_raster_name)

    # Get the remap table path
    remap_table_path = os.path.join(remap_table_gdb, remap_table)

    # Parse the geodatabase table and create a remap object
    remap_ranges = []
    with arcpy.da.SearchCursor(remap_table_path, ["FromValue", "ToValue", "OutputValue"]) as cursor:
        for row in cursor:
            remap_ranges.append(row)

    # Create a RemapRange object
    remap = arcpy.sa.RemapRange(remap_ranges)

    # Get Euclidean distance raster
    arcpy.gp.EucDistance_sa(feature_class, euclidean_raster, "")

    # Reclassify euclidean distance raster
    reclassified_raster = Reclassify(in_raster = euclidean_raster, reclass_field = "VALUE", remap = remap, missing_values = "DATA")
    reclassified_raster.save(reclassified_raster_path)

In [7]:
# grab all feature classes from a folder or geodatabase
def get_feature_classes(workspace):
    # Get the current workspace
    previous_workspace = arcpy.env.workspace

    try:
        # Set the new workspace
        arcpy.env.workspace = workspace

        # Get a list of all feature classes in the workspace
        feature_classes = arcpy.ListFeatureClasses()

        # Return the list of feature classes
        return feature_classes

    finally:
        # Reset the workspace to the previous workspace
        arcpy.env.workspace = previous_workspace

In [8]:
# grab all rasters from a given folder folder or geodatabase
def get_rasters(workspace):
    # Get the current workspace
    previous_workspace = arcpy.env.workspace

    try:
        # Set the new workspace
        arcpy.env.workspace = workspace
        
        # define wildcard parameter
        wildCard = "*reclass.tif" # alter according to your naming conventions

        # Get a list of all feature classes in the workspace
        rasters = arcpy.ListRasters(wildCard)

        # Return the list of feature classes
        return rasters

    finally:
        # Reset the workspace to the previous workspace
        arcpy.env.workspace = previous_workspace

In [22]:
def convert_to_one_bit(input_raster, output_raster):
    try:
        # Create a raster object from the input path
        input_raster_obj = arcpy.sa.Raster(input_raster)

        # Reclassify the input raster to binary (one-bit) format
        binary_raster = arcpy.sa.Con(input_raster_obj > 0, 1)

        # Set the pixel type of the output raster to 1_BIT
        arcpy.management.CopyRaster(binary_raster, output_raster, pixel_type="1_BIT")

        print(f"Conversion to one-bit raster completed successfully. Result saved to {output_raster}")

    except arcpy.ExecuteError:
        print(arcpy.GetMessages())
    except Exception as e:
        print(f"An error occurred: {str(e)}")

#### Generate reclassified euclidean distance rasters

In [10]:
# get feature classes for euclidean distance calculation
feature_classes = get_feature_classes(input_gdb)
feature_classes

['airfields',
 'airports',
 'cities',
 'lakes',
 'military',
 'mines',
 'railroads',
 'rivers']

In [12]:
# Iterate through the feature classes and perform Euclidean distance and reclassification
for feature_class in feature_classes:
    # Specify the remap table corresponding to the feature class
    remap_table = feature_class + "_remap_table"  # Adjust this based on your remap table naming convention

    # Call the function to perform Euclidean distance and reclassification
    euclidean_reclassify(feature_class, remap_table)

#### Build and execute raster calculator expression to generate technical inclusion layer

In [11]:
# Get a list of reclassified rasters in the scratch folder
reclassified_rasters = get_rasters(scratch_folder)
reclassified_rasters

['airfields_reclass.tif',
 'airports_reclass.tif',
 'cities_reclass.tif',
 'lakes_reclass.tif',
 'military_reclass.tif',
 'mines_reclass.tif',
 'railroads_reclass.tif',
 'rivers_reclass.tif']

In [12]:
# Build the raster calculator expression to calculate the final output raster
expression = "Con({condition}, 1)".format(
    condition=" & ".join([f'arcpy.sa.EqualTo("{raster}", 1)' for raster in reclassified_rasters])
)
expression

'Con(arcpy.sa.EqualTo("airfields_reclass.tif", 1) & arcpy.sa.EqualTo("airports_reclass.tif", 1) & arcpy.sa.EqualTo("cities_reclass.tif", 1) & arcpy.sa.EqualTo("lakes_reclass.tif", 1) & arcpy.sa.EqualTo("military_reclass.tif", 1) & arcpy.sa.EqualTo("mines_reclass.tif", 1) & arcpy.sa.EqualTo("railroads_reclass.tif", 1) & arcpy.sa.EqualTo("rivers_reclass.tif", 1), 1)'

In [14]:
# Set the output path for the final raster
final_raster = os.path.join(results_folder, "technicalInclusionArea_Wind.tif")
final_raster_binary = os.path.join(results_folder, "test_binary.tif")

In [15]:
arcpy.env.workspace = scratch_folder

In [18]:
# Perform the raster calculator operation
arcpy.gp.RasterCalculator_sa(expression, final_raster)

<geoprocessing server result object at 0x257774c26c0>

In [23]:
convert_to_one_bit(final_raster, final_raster_binary)

Conversion to one-bit raster completed successfully. Result saved to C:\Users\Zachary\ASSET\supplyCurve\analysis\data\technicalInclusionAreas\test_binary.tif
